In [2]:
# JSON 파일 로드
with open("card_infos_short.json", "r", encoding="utf-8") as f:
    card_data = json.load(f)

In [4]:
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
import json

documents = []

for card in card_data:
    content_blocks = []

    # 카드 기본 정보 추가
    content_blocks.append(f"카드이름: {card['카드이름']}")
    content_blocks.append(f"카드사: {card['카드사']}")
    content_blocks.append(f"카드브랜드: {card['카드브랜드']}")
    content_blocks.append(f"국내연회비: {card['국내연회비']}원")
    content_blocks.append(f"해외연회비: {card['해외연회비']}원")
    # content_blocks.append(f"카드 URL: {card['카드_url']}")

    # 주요 혜택 (유의사항 제외)
    benefit_texts = []
    for section, content in card["주요혜택"].items():
        if section.strip() != "유의사항":
            benefit_texts.append(f"[{section}] {content}")

    if benefit_texts:
        content_blocks.append("\n\n".join(benefit_texts))

    # 최종 page_content
    final_text = "\n\n".join(content_blocks).replace("\n", " ")

    # Document 생성
    doc = Document(
        page_content=final_text,
        metadata={
            # "카드_url": card["카드_url"],
            "카드이름": card["카드이름"],
            "카드사": card["카드사"],
            "카드브랜드": card["카드브랜드"],
            "국내연회비": card["국내연회비"],
            "해외연회비": card["해외연회비"],
            "원본_전체_혜택": json.dumps(card["주요혜택"], ensure_ascii=False)
        }
    )

    documents.append(doc)


In [6]:
documents[1].metadata

{'카드이름': '신한카드 B.Big(삑)',
 '카드사': '신한카드',
 '카드브랜드': 'VISA',
 '국내연회비': 0,
 '해외연회비': 10000,
 '원본_전체_혜택': '{"대중교통": "버스, 지하철 후불 교통 이용 금액 일 200원 ~ 일 600원 할인", "택시": "택시/KTX 이용 금액 10% 할인", "백화점": "4대 백화점 5% 할인", "카페": "커피전문점 10% 할인", "편의점": "편의점 5% 할인", "통신": "이동통신요금 5% 할인", "영화": "CGV, 메가박스", "캐시백": "전기요금, 통신요금, 해외이용금액 5% 캐시백 제공", "유의사항": "- 본 카드의 서비스 내용은 당사 및 제휴사 사정에 따라 사전 고지 후 변경 또는 중단될 수 있으며, 카드 이용금액에 따라 제한될 수 있습니다.- 별도의 이벤트를 진행하지 않는 한 타 쿠폰 및 타 카드(신용카드, 통신계 카드 등) 타 할인혜택의 중복 사용은 불가합니다.- 계약 체결 전 상품에 관한 사항은 상품설명서 및 약관을 읽어보시기 바랍니다.※ 카드이용시 제공되는 포인트 및 할인혜택 등의 부가서비스는 카드 신규출시(2020.11.10) 이후 3년 이상 축소, 폐지 없이 유지됩니다.※ 상기에도 불구하고, 다음과 같은 사유가 발생한 경우 카드사는 부가서비스를 변경할 수 있습니다.     ① 카드사의 휴업·파산·경영상의 위기 등에 따른 불가피한 경우     ② 제휴업체의 휴업•파산•경영상의 위기로 인해 불가피하게 부가서비스를 축소•변경하는 경우로서 다른 제휴업체를 통해 동종의 유사한 부가서비스 제공이 불가한 경우     ③ 제휴업체가 카드사의 의사에 반하여 해당 부가서비스를 축소하거나 변경 시, 당초 부가서비스에 상응하는 다른 부가서비스를 제공하는 경우     ④ 부가서비스를 3년 이상 제공한 상태에서 해당 부가서비스로 인해 카드의 수익성이 현저히 낮아진 경우※ 카드사가 부가서비스를 변경하는 경우에는 부가서비스 변경 사유, 변경 내용 등을 사유발생 즉시 아래 고지방법 

In [8]:
documents[1187].page_content

'카드이름: 삼성 iD ON 카드  카드사: 삼성카드  카드브랜드: Mastercard  국내연회비: 20000원  해외연회비: 20000원  [할인] 온라인 간편결제·해외 3%·1% 할인  [생활] 교통·이동통신·스트리밍 10% 할인  [선택형] 내 마음대로 고르는 디자인- 다양한 스타일로 새롭게 선보이는 삼성 iD 카드를 소개합니다.'

In [45]:
import tiktoken

# 사용할 모델의 토크나이저 로드
enc = tiktoken.encoding_for_model("text-embedding-3-small")

def count_tokens(text):
    tokens = enc.encode(text)
    return len(tokens)

# documents는 langchain Document 객체 리스트라고 가정
total_tokens = sum(count_tokens(doc.page_content) for doc in documents)
print(f"총 토큰 수: {total_tokens}")

# 총 토큰 수: 5675318
# 총 토큰 수: 3558221
# 총 토큰 수: 411322

총 토큰 수: 356478


In [9]:
documents[:2]

[Document(metadata={'카드이름': '신한카드 The CLASSIC-Y', '카드사': '신한카드', '카드브랜드': 'VISA', '국내연회비': 0, '해외연회비': 100000, '원본_전체_혜택': '{"선택형": "Gift Option Service - 매년 1회 옵션 품목 중 한 가지를 선택하여 이용", "모든가맹점": "일시불/할부 이용금액의 0.7% 마이신한포인트 적립\\n해외, 명품 아울렛(신세계/롯데), 면세점, 골프 업종 이용 시 5% 마이신한포인트 적립", "주유소": "GS칼텍스 주유 시 리터당 60원 결제일 할인 ", "생활": "커피/제과/택시/화장품/영화 5% 할인 ", "면세점": "제주 JDC 면세점 이용 시 8% 할인", "진에어": "진에어 국제선 항공권 결제일 할인", "공연/전시": "오디오 가이드 및 오페라 글라스 무료 이용 서비스", "프리미엄": "국제브랜드사 서비스 공통사항 안내", "유의사항": "- 계약 체결 전 상품에 관한 사항은 상품설명서 및 약관을 읽어보시기 바랍니다.※ 카드이용시 제공되는 포인트 및 할인혜택 등의 부가서비스는 카드 신규출시(2014.10.31) 이후 3년 이상 축소, 폐지 없이 유지됩니다.※ 상기에도 불구하고, 다음과 같은 사유가 발생한 경우 카드사는 부가서비스를 변경할 수 있습니다.     ① 카드사의 휴업·파산·경영상의 위기 등에 따른 불가피한 경우     ② 제휴업체의 휴업•파산•경영상의 위기로 인해 불가피하게 부가서비스를 축소•변경하는 경우로서 다른 제휴업체를 통해 동종의 유사한 부가서비스 제공이 불가한 경우     ③ 제휴업체가 카드사의 의사에 반하여 해당 부가서비스를 축소하거나 변경 시, 당초 부가서비스에 상응하는 다른 부가서비스를 제공하는 경우     ④ 부가서비스를 3년 이상 제공한 상태에서 해당 부가서비스로 인해 카드의 수익성이 현저히 낮아진 경우※ 카드사가 부가서비스를 변경하는 경우에는 부가서비스 변경 사유, 변경 내용 등을 사유발생 즉시 아래 고지방법 중

In [15]:
!pip install -U langchain-chroma

In [10]:
from langchain_openai import OpenAIEmbeddings  # 또는 HuggingFaceEmbeddings
from langchain_chroma import Chroma
from dotenv import load_dotenv
load_dotenv()

True

In [8]:
embedding_model = OpenAIEmbeddings(model = "text-embedding-3-large")

# Chroma 벡터스토어 만들기
vectorstore = Chroma.from_documents(
    documents,
    embedding_model,
    persist_directory="chroma_card_db"  # 저장 경로
)

In [11]:
# 저장된 Chroma 벡터스토어 불러오기
vectorstore = Chroma(
    persist_directory="chroma_card_db",
    embedding_function=embedding_model
)

In [36]:
query = "삼성카드의 연회비 조건이 좋은 카드를 알려주세요."
retrieved_docs = vectorstore.similarity_search_with_score(query, k=5)

In [42]:
retrieved_docs[0]

(Document(id='92343a2d-478f-4905-89bd-edcfcfd56c2a', metadata={'카드사': '삼성카드', '카드이름': 'KDB SAMSUNGCARD 4', '국내연회비': 5000, '해외연회비': 0, '카드브랜드': '없음', '원본_전체_혜택': '{"모든가맹점": "0.7% 결제일할인(청구할인)", "영화": "모든 영화관 2,500원 결제일할인", "은행사": "KDB산업은행 금융/여행/생활 서비스", "프리미엄": "American Express Selects 서비스", "경기관람": "프로스포츠 프로모션 서비스", "유의사항": "유의사항- 본 카드의 서비스 내용은 당사 및 제휴사 사정에 따라 사전 고지 후 변경 또는 중단될 수 있으며, 카드 이용금액에 따라 제한될 수 있습니다.- 별도의 이벤트를 진행하지 않는 한 타 쿠폰 및 타 카드(신용카드, 통신계 카드 등) 타 할인혜택의 중복 사용은 불가합니다.- 계약 체결 전 상품에 관한 사항은 상품설명서 및 약관을 읽어보시기 바랍니다.- 카드 이용 시 제공되는 포인트 및 할인혜택 등의 부가서비스는 카드 신규출시 이후 3년 이상 축소·폐지 없이 유지 됩니다.- 상기에도 불구하고, 다음과 같은 사유가 발생한 경우 카드사는 부가서비스를 변경할 수 있습니다.  ① 카드사 또는 부가서비스 관련 제휴업체의 휴업·도산·경영위기, 천재지변, 금융환경 급변 또는 그 밖에 이에 준하는 사유의 발생  ② 카드사의 노력에도 제휴업체가 일방적으로 부가서비스 변경을 통보(단, 다른 제휴업체를 통해 동종의 유사한 부가서비스 제공이 가능한 경우 제외)  ③ 카드 신규출시 이후 3년 이상 경과했고, 해당 카드의 수익성 유지가 어려운 경우- 카드사가 부가서비스를 변경하는 경우에는 부가서비스 변경 사유, 변경 내용 등을 사유발생 즉시 홈페이지에 게시하고, 개별 고지해 드립니다. 특히 카드 신규 출시 이후 3년 이상 경과했고, 해당 카드의 수익성 유지가 어려워져 부가서비스를 변경하는 경우에는 6개월 전부

In [12]:
# Retriever 생성
retriever = vectorstore.as_retriever(
    search_kwargs={"k": 3}  # Top 3 유사 Document 가져오게 설정
)

In [13]:
retriever.invoke("삼성카드를 추천해줘")

[Document(id='bbfdda9a-a62d-47bb-a1ad-46c1f25771fe', metadata={'국내연회비': 20000, '해외연회비': 20000, '카드이름': '(심의용) 삼성카드 신규상품1', '카드브랜드': 'Mastercard', '원본_전체_혜택': '{"적립": "생활 필수 영역 0.5% KTX 마일리지 포인트 추가 적립", "기타": "특수 소재 카드", "유의사항": "KTX 마일리지 포인트 안내 - KTX 마일리지 포인트는 KTX 삼성카드 이용 시 적립되는 포인트- KTX 마일리지 포인트는 ‘KTX 마일리지’로 자동 전환 후 한국철도공사 홈페이지 또는 코레일톡 앱에서 조회 및 사용 가능- ‘KTX 마일리지’로 전환된 KTX 마일리지 포인트는 전환 취소 불가 (1 KTX 마일리지 포인트 = 1 KTX 마일리지의 가치) - 매월 1일~말일까지의 매출전표 접수건에 대해 적립된 KTX 마일리지 포인트는 다음달 20일까지 접수된 매출전표 취소건을 반영하여 다음달 25일에 ‘KTX 마일리지’로 자동 전환되며, 아래의 경우 해당월 전환 불가* 코레일멤버십 회원이 아닌 경우 - 코레일멤버십 회원 자격을 상실한 경우 - 적립된 KTX 마일리지 포인트가 0 포인트 이하인 경우* ‘KTX 마일리지’ 전환 시점에 통신 장애가 발생한 경우 * ‘KTX 마일리지’로 전환되지 않은 KTX 마일리지 포인트는 전환조건 충족 시 다음달 25일에 전환- 삼성카드의 다른 포인트와 합산하여 사용 불가 * KTX 마일리지 포인트의 유효기간은 5년으로, 유효기간 만료 시 월 단위로 자동 소멸 - KTX 마일리지 포인트 적립에 대한 자세한 내용은 삼성카드 홈페이지(PC) 또는 앱에서 확인 - ‘KTX 마일리지’로 전환된 이후, 해당 결제건 취소 시 자세한 내용은 ‘안내사항’에서 확인 KTX 마일리지 안내- ‘KTX 마일리지’란 철도승차권 구매 시 사용할 수 있는 결제 수단으로, 한국철도공사 홈페이지 또는 코레일톡 앱에서 코레일멤버십 가입 후 적립 및 사용 가능- 

In [14]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain_community.cross_encoders import HuggingFaceCrossEncoder

In [15]:
# cross encoder (embedding) model이 필요행! - cohere : embedding쪽이 성능이 좋음. 근데 오픈소스 없삼
reranker_model_id = "BAAI/bge-reranker-v2-m3"

retriever = vectorstore.as_retriever(
    search_kwargs={"k": 3}  # Top 3 유사 Document 가져오게 설정
)
reranker = HuggingFaceCrossEncoder(model_name = reranker_model_id)
compressor = CrossEncoderReranker(model = reranker, top_n= 5)		# 압축방식 객체 생성 <- crossencoder 객체
reranker_retriever = ContextualCompressionRetriever(
    base_retriever=retriever,	# 처음 문서를 조회할 retriever
    base_compressor=compressor,	# base retriever가 검색한 문서를 압축하는 알고리즘, (Reranker)
)

In [ ]:
query = "삼성카드의 연회비 조건이 좋은 카드를 알려주세요."
base_result = retriever.invoke(query)
rerank_result = reranker_retriever.invoke(query)
base_result_str = [d.page_content for d in base_result]
rerank_result_str = [d.page_content for d in rerank_result]

In [18]:
base_result_str

['카드이름: KDB SAMSUNGCARD 4  카드사: 삼성카드  카드브랜드: 없음  국내연회비: 5000원  해외연회비: 0원  [모든가맹점] 0.7% 결제일할인(청구할인)  [영화] 모든 영화관 2,500원 결제일할인  [은행사] KDB산업은행 금융/여행/생활 서비스  [프리미엄] American Express Selects 서비스  [경기관람] 프로스포츠 프로모션 서비스',
 '카드이름: 삼성 BIZ iD BENEFIT카드  카드사: 삼성카드  카드브랜드: Mastercard  국내연회비: 30000원  해외연회비: 30000원  [비즈니스] 4대 사회보험, 도시가스요금, 전기요금, 할인점, 온라인쇼핑몰, 식자재몰, 해외 1.5% 결제 할인  [모든가맹점] 국내 가맹점 0.5% 결제일할인  [생활] 주유·전기차 충전요금·이동통신·인터넷/유선통신·렌탈·보안·방역 3% 결제일할인  [공과금/렌탈] 세무지원 서비스  [선택형] 원하는 플레이트 디자인 선택 가능',
 '카드이름: 신세계인터내셔날 삼성카드  카드사: 삼성카드  카드브랜드: Mastercard  국내연회비: 25000원  해외연회비: 25000원  [쇼핑] 신세계인터내셔날 브랜드·신세계백화점 1%·2% 결제일할인  [카페] 스타벅스 20% 결제일할인  [간편결제] 온라인 간편결제 1% 결제일할인  [해외] 해외 1.5% 결제일할인  [기타] 신세계백화점 제휴 서비스  [프리미엄 서비스] Mastercard 프리미엄 서비스  [공항라운지] 인천·김포·김해공항 라운지 본인 이용 무료']

In [19]:
rerank_result_str

['카드이름: 삼성 BIZ iD BENEFIT카드  카드사: 삼성카드  카드브랜드: Mastercard  국내연회비: 30000원  해외연회비: 30000원  [비즈니스] 4대 사회보험, 도시가스요금, 전기요금, 할인점, 온라인쇼핑몰, 식자재몰, 해외 1.5% 결제 할인  [모든가맹점] 국내 가맹점 0.5% 결제일할인  [생활] 주유·전기차 충전요금·이동통신·인터넷/유선통신·렌탈·보안·방역 3% 결제일할인  [공과금/렌탈] 세무지원 서비스  [선택형] 원하는 플레이트 디자인 선택 가능',
 '카드이름: 신세계인터내셔날 삼성카드  카드사: 삼성카드  카드브랜드: Mastercard  국내연회비: 25000원  해외연회비: 25000원  [쇼핑] 신세계인터내셔날 브랜드·신세계백화점 1%·2% 결제일할인  [카페] 스타벅스 20% 결제일할인  [간편결제] 온라인 간편결제 1% 결제일할인  [해외] 해외 1.5% 결제일할인  [기타] 신세계백화점 제휴 서비스  [프리미엄 서비스] Mastercard 프리미엄 서비스  [공항라운지] 인천·김포·김해공항 라운지 본인 이용 무료',
 '카드이름: KDB SAMSUNGCARD 4  카드사: 삼성카드  카드브랜드: 없음  국내연회비: 5000원  해외연회비: 0원  [모든가맹점] 0.7% 결제일할인(청구할인)  [영화] 모든 영화관 2,500원 결제일할인  [은행사] KDB산업은행 금융/여행/생활 서비스  [프리미엄] American Express Selects 서비스  [경기관람] 프로스포츠 프로모션 서비스']

In [20]:
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
# RAG용 prompt template : "답변을 context 기반으로 해야한다"라는 system message가 들어가야함.
template = """
# Instruction:
당신은 정확한 정보 제공을 우선시하는 인공지능 어시스턴트입니다.
주어진 Context에 포함된 정보만 사용해서 질문에 답변하세요.
Context에 질문에 대한 명확한 정보가 있는 경우 그 내용을 바탕으로 답변하세요.
Context에 질문에 대한 명확한 정보없을 경우 "정보가 부족해서 답을 알 수 없습니다." 라고 대답합니다.
절대 Context에 없는 내용을 추측하거나 일반 상식을 이용해 답을 만들어서 대답하지 않습니다.

Context:
{context}

질문:
{query}
"""
prompt_template = PromptTemplate(template=template)

In [27]:
from langchain_openai import ChatOpenAI

# rerank, prompt 다듬기 해보기
llm_model = ChatOpenAI(model_name = "gpt-4.1")

from langchain_core.runnables import chain
@chain
def RAG_Rerank_chain(query : str):
    docs = reranker_retriever.get_relevant_documents(query)
    page_contents = [doc.page_content for doc in docs]
    prompt = prompt_template.invoke({"context" : page_contents, "query" : query})
    result = llm_model.invoke(prompt)
    return StrOutputParser().invoke(result)

In [28]:
result = RAG_Rerank_chain.invoke("삼성카드의 교통 혜택이 좋은 카드를 추천해줘")

In [30]:
print(result)

Context에 따르면, 교통 혜택이 좋은 삼성카드로는 다음 두 가지가 있습니다.

1. iD MOVE카드: 대중교통·택시 10% 결제일 할인 혜택이 있습니다.
2. SC제일은행 삼성체크카드 YOUNG: 대중교통 이용금액의 10% 캐시백 혜택과, 후불교통카드 기능을 제공합니다.

따라서, 대중교통 관련 혜택이 좋은 삼성카드를 원한다면 위 두 카드가 적합합니다.


In [44]:
from ragas.metrics import faithfulness, answer_relevancy, context_precision, context_recall
from ragas import evaluate
from datasets import Dataset

# 1. 체인 실행해서 답변 얻기
query = "해외 수수료 우대되는 카드 추천해줘"
docs = reranker_retriever.get_relevant_documents(query)
contexts = [doc.page_content for doc in docs]

# 기존 RAG 답변
prompt = prompt_template.invoke({"context": contexts, "query": query})
answer = llm_model.invoke(prompt).content

# 2. 평가용 데이터셋 생성
rag_eval_data = Dataset.from_dict({
    "question": [query],
    "contexts": [contexts],
    "answer": [answer],
    "ground_truth": ["여기에 정답 텍스트 넣기 (없으면 그냥 빈 문자열 '')"]
})

# 3. RAGAS 평가 실행
result = evaluate(
    rag_eval_data,
    metrics=[
        faithfulness,
        answer_relevancy,
        context_precision,
        context_recall
    ]
)

print(result)


Evaluating:   0%|          | 0/4 [00:00<?, ?it/s]

The LLM did not return a valid classification.


{'faithfulness': 0.7143, 'answer_relevancy': 0.8281, 'context_precision': 1.0000, 'context_recall': nan}


## 모든 세부 혜택 크롤링한 data로 진행하던 흔적

In [ ]:
# 신규발급중지, 브랜드 추가
import requests
import re
import json
import pandas as pd
from bs4 import BeautifulSoup
from tqdm import tqdm

def extract_card_info(card_id):
    url = f"https://api.card-gorilla.com:8080/v1/cards/{card_id}"
    headers = {"User-Agent": "Mozilla/5.0"}
    res = requests.get(url, headers=headers)

    if res.status_code != 200:
        return None

    data = res.json()

    # 카드 기본 정보
    card_idx = data.get("idx")
    card_name = data.get("name")

    # 신규 발급 중단 여부
    is_discon = data.get("is_discon", False)
    if is_discon:
        return None

    card_company = data.get("corp", {}).get("name")

    # 카드 브랜드
    brands = data.get("brand", [])
    card_brand = brands[0]["name"] if brands else "없음"

    # 국내/해외 연회비 추출
    annual_fee_detail = data.get("annual_fee_detail") or ""
    domestic_fee = re.search(r'국내.*?([0-9,]+)원', annual_fee_detail)
    domestic_fee = int(domestic_fee.group(1).replace(",", "")) if domestic_fee else 0
    foreign_fee = re.search(r'해외.*?([0-9,]+)원', annual_fee_detail)
    foreign_fee = int(foreign_fee.group(1).replace(",", "")) if foreign_fee else 0

    # 카드 혜택
    benefits_dict = {}
    key_benefit = data.get("key_benefit", [])
    for benefit in key_benefit:
        title = benefit.get("title")
        comment = benefit.get("comment") or ""
        info = benefit.get("info") or ""
        
        # 전체 내용 추출 (혜택 제목 + 세부 정보 모두)
        if comment:
            if info:
                soup = BeautifulSoup(info, "html.parser")
                desc = soup.get_text().strip().replace('\xa0', ' ')  # 줄바꿈을 공백으로 대체
                
            elif comment:
                desc = comment
            else:
                desc = "내용 없음"
     
        benefits_dict[title] = desc

    return {
        "카드_url": "https://www.card-gorilla.com/card/detail/" + str(card_idx),
        "카드이름": card_name,
        "카드사": card_company,
        "카드브랜드": card_brand,
        "국내연회비": domestic_fee,
        "해외연회비": foreign_fee,
        "주요혜택": benefits_dict
    }

# 수집할 카드 ID 범위 지정 (예: 1~2857)
card_infos = []
for card_id in tqdm(range(1, 2858)):
    info = extract_card_info(card_id)
    if info:
        card_infos.append(info)

100%|██████████| 2857/2857 [05:23<00:00,  8.84it/s]


In [66]:
with open("card_infos.json", "w", encoding="utf-8") as f:
    json.dump(card_infos, f, ensure_ascii=False, indent=4)

In [95]:
# CSV로 저장
df = pd.DataFrame(card_infos)
df.to_csv("card_info.csv", index=False, encoding="utf-8-sig")

In [78]:
type(card_data)

list

In [118]:
from langchain_core.documents import Document

def make_documents(card_json_list):
    documents = []
    for card in card_json_list:
        metadata = {
            "카드이름": card.get("카드이름"),
            "카드사": card.get("카드사"),
            "카드브랜드": card.get("카드브랜드"),
            "국내연회비": card.get("국내연회비"),
            "해외연회비": card.get("해외연회비"),
            # "카드_url": card.get("카드_url")  # 필요 시 URL도 메타에 넣기
        }

        # 주요혜택 내용을 긴 텍스트로 합치기
        content_parts = []
        혜택_dict = card.get("주요혜택", {})
        for category, desc in 혜택_dict.items():
            if desc:
                content_parts.append(f"[{category}]\n{desc.strip()}\n")
        content = "\n".join(content_parts)

        # Document 객체 생성
        doc = Document(
            page_content=content,
            metadata=metadata
        )
        documents.append(doc)

    return documents

# JSON 파일 로드
with open("card_infos.json", "r", encoding="utf-8") as f:
    card_data = json.load(f)

documents = make_documents(card_data)

In [119]:
documents[0].page_content

'[선택형]\nGift Option 서비스는 매년 1회 아래 품목 중 한 가지를 선택하여 이용하실 수 있습니다.- 포인트 : 마이신한포인트 적립(7만점) / 1년1회- 문화 : 문화상품권(8만원) / 1년1회- 요식 : 패밀리 레스토랑 11만원 이용권 / 1년1회- 호텔 : 호텔 애프터눈 티 SET 이용권 / 1년2회Gift Option 서비스 이용방법- 연회비 청구 기준으로 1년간 Gift Option 중 택 1하여 지정된 서비스에 한해 신청 가능하며, 신청기간 경과 이후 미사용 건에 대해서는 소급 신청이 불가합니다.- Gift Option 서비스는 The CLASSIC-Y 카드 발급 초년도의 경우 연회비 납부 및 카드 일시불+할부 이용금액 20만원 이상이용 시 신청 가능하며,  2차년도부터는 연회비 납부 및 전년도 카드 일시불+할부 이용금액 300만원 이상 이용 시 신청 가능합니다.- 서비스 신청 시 본인이 수령을 원하는 자택 또는 청구지 주소로 이용권 또는 신청 물품을 수령하시고 사용하시면 됩니다. (Option 수령 기간 : 약 10일)- Gift Option 서비스는 액면에 기재되어 있는 기간에 한하여 사용이 가능합니다.- Gift Option 서비스 취소 및 변경은 신청 당일에 한하여 가능합니다.- Gift Option 서비스는 가족카드 회원에게는 제공되지 않습니다.Option 1. 마이신한포인트 적립(7만점)- 사용방법(자세한 내용은 신한카드 홈페이지 : www.shinhancard.com에서 확인 가능합니다.)  * 1) 1Point 이상 : 마이신한포인트 가맹점, MY신한몰, 연회비 결제, 포인트 기부, 지방세 납부, 기프트카드 구매  * 2) 200Point 이상 : SMS 수수료 결제  * 3) 10,000Point 이상 : 포인트 캐시백- 적립기간 : 적립일 기준으로 5년- 사용제한 : 카드 해지 등의 사유로 거래정지가 된 경우- 유효기간 및 소멸 : 적립된 마이신한포인트의 유효기간은 적립일로부터 60개월이며, 유효기간이 경과된 마이신한포

In [121]:
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter

documents = []

for card in card_infos:
    benefit_texts = []

    for section, content in card["주요혜택"].items():
        # 딱 key가 "유의사항"인 것만 제외
        if section.strip() != "유의사항":
            benefit_texts.append(f"[{section}] {content}")

    final_text = "\n\n".join(benefit_texts)

    doc = Document(
        page_content=final_text,
        metadata={
            "카드이름": card["카드이름"],
            "카드사": card["카드사"],
            "카드브랜드": card["카드브랜드"],
            "국내연회비": card["국내연회비"],
            "해외연회비": card["해외연회비"],
            "원본_전체_혜택": json.dumps(card["주요혜택"], ensure_ascii=False)  # 최종 출력용
        }
    )

    documents.append(doc)

In [113]:
print(documents[0])

page_content='[선택형] Gift Option 서비스는 매년 1회 아래 품목 중 한 가지를 선택하여 이용하실 수 있습니다.- 포인트 : 마이신한포인트 적립(7만점) / 1년1회- 문화 : 문화상품권(8만원) / 1년1회- 요식 : 패밀리 레스토랑 11만원 이용권 / 1년1회- 호텔 : 호텔 애프터눈 티 SET 이용권 / 1년2회Gift Option 서비스 이용방법- 연회비 청구 기준으로 1년간 Gift Option 중 택 1하여 지정된 서비스에 한해 신청 가능하며, 신청기간 경과 이후 미사용 건에 대해서는 소급 신청이 불가합니다.- Gift Option 서비스는 The CLASSIC-Y 카드 발급 초년도의 경우 연회비 납부 및 카드 일시불+할부 이용금액 20만원 이상이용 시 신청 가능하며,  2차년도부터는 연회비 납부 및 전년도 카드 일시불+할부 이용금액 300만원 이상 이용 시 신청 가능합니다.- 서비스 신청 시 본인이 수령을 원하는 자택 또는 청구지 주소로 이용권 또는 신청 물품을 수령하시고 사용하시면 됩니다. (Option 수령 기간 : 약 10일)- Gift Option 서비스는 액면에 기재되어 있는 기간에 한하여 사용이 가능합니다.- Gift Option 서비스 취소 및 변경은 신청 당일에 한하여 가능합니다.- Gift Option 서비스는 가족카드 회원에게는 제공되지 않습니다.Option 1. 마이신한포인트 적립(7만점)- 사용방법(자세한 내용은 신한카드 홈페이지 : www.shinhancard.com에서 확인 가능합니다.)  * 1) 1Point 이상 : 마이신한포인트 가맹점, MY신한몰, 연회비 결제, 포인트 기부, 지방세 납부, 기프트카드 구매  * 2) 200Point 이상 : SMS 수수료 결제  * 3) 10,000Point 이상 : 포인트 캐시백- 적립기간 : 적립일 기준으로 5년- 사용제한 : 카드 해지 등의 사유로 거래정지가 된 경우- 유효기간 및 소멸 : 적립된 마이신한포인트의 유효기간은 적립일로부터 60개월이며, 유효기

In [97]:
from dotenv import load_dotenv
load_dotenv()

True

In [122]:
import tiktoken

# 사용할 모델의 토크나이저 로드
enc = tiktoken.encoding_for_model("text-embedding-3-small")

def count_tokens(text):
    tokens = enc.encode(text)
    return len(tokens)

# documents는 langchain Document 객체 리스트라고 가정
total_tokens = sum(count_tokens(doc.page_content) for doc in documents)
print(f"총 토큰 수: {total_tokens}")

# 총 토큰 수: 5675318, 
# 총 토큰 수: 3558221

총 토큰 수: 3558221


In [114]:
from langchain_openai import OpenAIEmbeddings  # 또는 HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

embedding_model = OpenAIEmbeddings(model = "text-embedding-3-small")

# Chroma 벡터스토어 만들기
vectorstore = Chroma.from_documents(
    documents,
    embedding_model,
    persist_directory="chroma_card_db"  # 저장 경로
)

# 실제 저장
vectorstore.persist()

BadRequestError: Error code: 400 - {'error': {'message': 'Requested 2213741 tokens, max 300000 tokens per request', 'type': 'max_tokens_per_request', 'param': None, 'code': 'max_tokens_per_request'}}

In [19]:
vectorstore